In [15]:

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/app')
sys.path.insert(0,'..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:

from audio.redis import *
from audio.audio import *
from audio.process import *

from pathlib import Path
import json

import uuid

import pandas as pd

import asyncio
import redis.asyncio as aioredis
from dataclasses import dataclass
import numpy as np

In [6]:
with open('../header', 'rb') as f:
    header = f.read()

In [90]:
header

b'\x1aE\xdf\xa3\x9fB\x86\x81\x01B\xf7\x81\x01B\xf2\x81\x04B\xf3\x81\x08B\x82\x84webmB\x87\x81\x04B\x85\x81\x02\x18S\x80g\x01\xff\xff\xff\xff\xff\xff\xff\x15I\xa9f\x99*\xd7\xb1\x83\x0fB@M\x80\x86ChromeWA\x86Chrome\x16T\xaek\xbf\xae\xbd\xd7\x81\x01s\xc5\x87\x82\xab\x13\xbdt\xe8\x1b\x83\x81\x02\x86\x86A_OPUSc\xa2\x93OpusHead\x01\x02\x00\x00\x80\xbb\x00\x00\x00\x00\x00\xe1\x8d\xb5\x84G;\x80\x00\x9f\x81\x02bd\x81 \x1fC\xb6u\x01\xff\xff\xff\xff\xff\xff\xff\xe7\x81\x00\xa3C\xf8\x81\x00\x00\x80\xff\x83\xfd:\xfc\x03v\x90\x94y\xfd\xf9\x1en\xf8W\x93\xf2\xb7\xa3\x83_\xfbS:9Gi\x0f\xa0\x9e+'

In [7]:

redis_inner_client = await get_inner_redis()
redis_stream_client = await get_stream_redis()
client_id = '851f343e-4954-4f0a-8835-9664cc91c181'

In [46]:
conns = await get_connection_queues(name = 'initialFeed_audio', pattern = "*",redis_client= redis_stream_client)
conns

,queue,length
0,initialFeed_audio:a10d4270-5661-4838-a183-bc5a...,193
1,initialFeed_audio:53b126f5-91d4-4bc6-9be7-a231...,1186
2,initialFeed_audio:58d5988b-5504-4eac-8482-0bb0...,26
3,initialFeed_audio:80338840-b168-4616-9bad-24e4...,9


In [47]:
client_id = '851f343e-4954-4f0a-8835-9664cc91c181'

In [78]:
conn_id = conns['queue'].iloc[3].split(':')[1]

In [79]:
conn = ApiConnection(conn_id,redis_stream_client)

In [50]:
await conn.check_open()

False

In [92]:
chunks = await conn.pop_multiple(1)

In [89]:
cleaned

'x1aE\\xdf\\xa3\\x9fB\\x86\\x81\\x01B\\xf7\\x81\\x01B\\xf2\\x81\\x04B\\xf3\\x81\\x08B\\x82\\x84webmB\\x87\\x81\\x04B\\x85\\x81\\x02\\x18S\\x80g\\x01\\xff\\xff\\xff\\xff\\xff\\xff\\xff\\x15I\\xa9f\\x99*\\xd7\\xb1\\x83\\x0fB@M\\x80\\x86ChromeWA\\x86Chrome\\x16T\\xaek\\xbf\\xae\\xbd\\xd7\\x81\\x01s\\xc5\\x87\\xf9\\x1ey%]$X\\x83\\x81\\x02\\x86\\x86A_OPUSc\\xa2\\x93OpusHead\\x01\\x02\\x00\\x00\\x80\\xbb\\x00\\x00\\x00\\x00\\x00\\xe1\\x8d\\xb5\\x84G;\\x80\\x00\\x9f\\x81\\x02bd\\x81 \\x1fC\\xb6u\\x01\\xff\\xff\\xff\\xff\\xff\\xff\\xff\\xe7\\x81\\x00\\xa3C\\x1b\\x81\\x00\\x00\\x80\\xff\\x83\\x02\\xff:\\xff\\xfe\\x7f\\xbc\\x8aBe\\xd8\\x1f6{O\\xeb:\\xf9\\x1d\\xbf\\x04\\\'r\\xfe#6\\x98\\xef\\xc4\\xda\\xa0\\x97L\\xa1\\x18\\xc9\\xe6\\x9d\\x917A\\xa4\\xc3Z\\xcc\\x17\\xdazJ\\xa7D,\\xc4H\\xc1g\\xa4\\xf7\\x8a\\xceb^vGy\\x18 cr>\\x98{h\\xa4\\xd0\\x96\\xc3w\\x0e\\x19\\\\\\xcc}\\xaa\\rGm\\xd3\\x8aU\\x88\\xdd\\x9e\\xa8\\xd1dr\\tN\\x03\\xbc_A\\x0f;\\xf8.\\xc7v\\x10\\x94T\\xecFE\\x10\\x19\\xc1\\x9a5Dm\\x95\\

In [124]:
ApiConnection??

Init signature: ApiConnection(conn_id, redis_client, is_video=False)
Source:        
class ApiConnection:
    """
    connection to initial chrome extention feed
    """
    def __init__(self, conn_id, redis_client,is_video=False):
        self.id = conn_id
        self.redis_client = redis_client
        self.is_video = is_video
        self.name = f"Connection:{self.id}"
        self.start_timestamp = pd.Timestamp.utcnow()
        self.queue = f"initialFeed_audio:{self.id}"
        self.__post_init__()

    def __post_init__(self):
        if self.is_video:self.queue = f"initialFeed:{self.id}"
        else:            self.queue = f"initialFeed_audio:{self.id}"

    async def set_open(self):
            self.open = True
            await self.set_update()

    async def set_closed(self):
        self.open = False
        await self.set_update()

    async def set_update(self):
        await self.redis_client.hset(self.name, 'open', int(self.open))

    async def enqueue(self, message

In [120]:
_, ch = await redis_stream_client.brpop(conns['queue'].iloc[1])

In [121]:
json.loads(ch['chunk']).

'{"chunk": "...", "timestamp": "2024-01-31 11:47:52.194702"}'

In [117]:
await conn.check_open()

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [111]:
new_items = await conn.pop_multiple(30)

In [112]:
new_items

In [ ]:
try:
    while True:
        print(start,end)
        audio_data = await audio_slicer.export_data(start,end)
        audio_name = str(uuid.uuid4())
        audio = Audio(chunk_name=audio_name, redis_client=redis_inner_client, data=audio_data)
        await audio.save()
        print('waiting diarize')
        diarization_result = await diarize(audio_data,client_id,audio_name,start)
        print('waiting transcribe')
        await transcribe(diarization_result,connection_id,audio_name)
        start_ = await get_next_chunk_start(diarization_result,length)
        start = start_ if start_ else start+length
        if end>audio_slicer.audio.duration_seconds:break
        end+=length
except KeyboardInterrupt:
    pass